In [2]:
import pandas as pd
from tfob import TFOb, get_bhsa

BHSA = get_bhsa()

ModuleNotFoundError: No module named 'tf'

In [2]:
from collections import Counter

### 1.Create two lists with the verses from MT Isaiah and 1QIsaa

#### 1.1 List of wanted motion verbs

In [3]:
motion_verbs = ['BW>[','HLK[','JY>[','JRD[','<BR[',
                '<LH[','CWB[','>TH[','BRX[','GJX[',
                'GLH[','GLL[','DXP[','DLG[','HWH=[',
                'XWC[','XLP[','XSH[','VB<[','VWF[',
                'MHR[','MWC[','NGC[','NHR[','NWX[',
                'NWS[','NXT[','NVP[','NS<[','NPL[',
                'NTK[','SBB[','SWR[','SLQ[','<WZ[',
                '<WP[','PNH[','PF<[','YWP[','Y<D[',
                'QHL[','QPY[','QRB[','RWY[','FVH[',
                'CWX[','CWR=[','CVP[','CQQ[','T>R[',
                'T<H[']

freq_verbs = ['BW>[','HLK[','JY>[','JRD[','<BR[',
                '<LH[','CWB[','>TH[','BRX[','GLH[',
                'HWH[','XLP[','XSH[','MHR[','NGC[',
                'NWX[','NWS[','NS<[','NPL[','NTK[',
                'SBB[','SWR[','<WP[','PNH[','PC<[',
                'QHL[','QRB[','RWY[','CVP[','T<H[']

#### 1.2 List of occurrences of the motion verbs (MT Isaiah)

In [36]:
verbs_bhsa = TFOb.all("word", BHSA).filter_in(lex=motion_verbs)
verbs_bhsa

<word_10834 "TWY> J<WPP TWY> J<LH JY> SBB SWBB HLK JNXHW JB> JPL JB>H MTHLK TLK CWBK TCWB JB> HBJ> JPLW NPLW [...] JY> HSB JB> H<BJRWNJ J<BJRHW JWLJKHW JSJRHW JSB JBJ>HW <LH HLJKW HBJ> JB>HW CWB <LWT J<L HBJ> JGL J<BR J<L">

In [46]:
verb = verbs_bhsa[0]
verb

<word_1 "TWY>">

In [53]:
for verb in verbs_bhsa:
    clause_phrases_functions = verb.to_clauses.to_phrases.function
    clause_phrases_functions
    if "Loca" in clause_phrases_functions:
        print(verb, verb.book[0], verb.chapter[0], verb.verse[0])

JTGL Genesis 9 21
JRY Genesis 18 2
GC Genesis 19 9
JS< Genesis 20 1
H<LHW Genesis 22 2
HCJBW Genesis 29 3
NGLW Genesis 35 7
J<L Genesis 35 13
HLKT Exodus 2 5
JB> Exodus 3 1
TQRB Exodus 3 5
NLKH Exodus 5 3
NLK Exodus 8 23
MBJ> Exodus 10 4
JNX Exodus 10 14
JWY> Exodus 11 4
JB>W Exodus 14 16
JB>W Exodus 14 22
B>JM Exodus 14 28
HLKW Exodus 14 29
HLKW Exodus 15 19
JRD Exodus 19 20
>BW> Exodus 20 24
T<LH Exodus 20 26
HTHLK Exodus 21 19
B>W Exodus 28 35
HQRBT Exodus 29 10
CWBW Exodus 32 27
><LH Exodus 33 3
><LH Exodus 33 5
JB> Exodus 38 7
MWSBT Exodus 39 13
J<L Exodus 40 25
HQRB Leviticus 6 7
HQRJB Leviticus 7 38
HWB> Leviticus 10 18
H<LH Leviticus 14 20
HBJ> Leviticus 14 23
HB>TJ Leviticus 26 36
HQRBM Numbers 3 4
JB> Numbers 6 10
JPL Numbers 14 5
JPLW Numbers 14 29
JPLW Numbers 14 32
JQHL Numbers 16 19
J<LW Numbers 16 27
JCB Numbers 17 15
HNXTM Numbers 17 19
JNX Numbers 17 22
HNJX Numbers 19 9
J<L Numbers 23 2
><L Numbers 23 4
J<L Numbers 23 14
J<L Numbers 23 30
JRD Numbers 34 11
HLK Deutero

In [57]:
#TESTING ZONE 
#verbs_bhsa[0]
#TFOb(215297, BHSA).to_verses.text
#TFOb(215297, BHSA).to_verses.pretty()
verb = verbs_bhsa[29]
cmpl = verb.to_clauses.to_phrases.filter_in(function=["Cmpl", "Loca"])
print(verb.book[0], verb.chapter[0], verb.verse[0])
print(verb, ",", verb.to_clauses)
print(cmpl.text, cmpl.to_words.prs)
print(cmpl.to_words.sp)
print(cmpl.to_phrases.det)
print(cmpl.to_words.nametype)

Genesis 6 20
JB>W , CNJM M KL JB>W >LJK
אֵלֶ֖יךָ  ['K']
['prep']
['det']
[None]


In [58]:
vb = [nametype for nametype in verbs_bhsa.to_verses.to_words.nametype if nametype]
print(len(vb))
print(len(verbs_bhsa.to_verses.to_words.filter_in(sp=["nmpr"])))

13616
12627


In [59]:
vb2 = [(word.sp[0], word.nametype[0]) for word in verbs_bhsa.to_clauses.to_phrases.filter_in(function=["Cmpl", "Loca"]).to_words.filter_in(sp=["subs", "nmpr"])]
Counter(vb2)

Counter({('subs', None): 6085,
         ('nmpr', 'topo'): 1310,
         ('nmpr', 'pers'): 769,
         ('nmpr', 'pers,gens,topo'): 257,
         ('nmpr', 'gens'): 11,
         ('nmpr', 'pers,gens'): 8,
         ('nmpr', 'topo,pers'): 2,
         ('nmpr', 'pers,god'): 1})

In [60]:
vb3 = [(word.sp[0], word.nametype[0]) for word in verbs_bhsa.to_clauses.to_phrases.filter_in(function=["Cmpl", "Loca"]).to_words.filter_in(sp=["subs", "nmpr", "prde", "prps", "prin"])]
Counter(vb3)

Counter({('subs', None): 6085,
         ('nmpr', 'topo'): 1310,
         ('nmpr', 'pers'): 769,
         ('nmpr', 'pers,gens,topo'): 257,
         ('prde', None): 114,
         ('nmpr', 'gens'): 11,
         ('prps', 'ppde'): 10,
         ('prin', None): 9,
         ('nmpr', 'pers,gens'): 8,
         ('nmpr', 'topo,pers'): 2,
         ('nmpr', 'pers,god'): 1})

### 2. Important function

In [61]:
# Create a dataset with the occurrences

def find_complements(verb):
    """Find the complement/locative of a verb. If no match, returns None"""
    if verb.source.name == "BHSA":
        complements = verb.to_clauses.to_phrases.filter_in(function=["Cmpl", "Loca"])
        return complements
    
    # if the verb is not BHSA, it's DSS
    verb_bhsa = find_bhsa_verb(verb)

    # Check if verses are identical
    if verb_bhsa and is_lex_identical(verb_bhsa.to_verses, find_dss_verse(verb)):
        return find_complements(verb_bhsa)
        
def find_cmpl_anim(cmpl):
    """Based on the nametype value, returns 'anim' or 'inanim' for one complement of a given verb. 
    If the nametype does not provide a clear distinction, returns 'check'.
    If there is no complement to the verb, returns None."""
    
    anim = ["pers", "gens", "god"]
    inanim = ["mens", "topo"]

    nouns = cmpl.to_words.filter_in(sp=["subs", "nmpr"])
    nouns_animacy = []
    for noun in nouns:
        nt = noun.nametype[0]
        if nt in anim:
            nouns_animacy.append("anim")
        elif nt in inanim:
            nouns_animacy.append("inanim")
    return " ".join(nouns_animacy)


def find_cmpl_individuation(cmpl):
    """Returns whether the nouns in a cmpl are substantives or proper nouns."""
    cmpl_indiv = []
    for word in cmpl.to_words.filter_in(sp=["subs","nmpr"]):
        cmpl_indiv.append(word.sp[0])
   
    # Handle cases when the individuation is a pronominal suffixe (prsf)    
    simplified_cmpl = []
   
    if len(cmpl.to_words) == 1:
        if cmpl.to_words.sp[0] == "prep":
            if cmpl.to_words.prs not in ["n/a", "absent"]:
                cmpl_indiv.append("prsf")
        else:
            cmpl_indiv.append(cmpl.to_words.sp[0])
        
    # cmpl_indiv empty string if empty    
    if len(cmpl_indiv) == 0:
        cmpl_indiv.append("")
        
    return cmpl_indiv[0]

def find_cmpl_nametype(cmpl):
    words = cmpl.to_words
    cmpl_nt = []
    for word in words:
        if word.nametype[0]:
            nt = word.nametype[0]
            cmpl_nt.append(nt)
            
    if find_cmpl_individuation(cmpl) == "prsf":
        cmpl_nt.append("prsf")
        
    return " ".join(cmpl_nt)


def find_cmpl_individuation(cmpl):
    """Returns whether the nouns in a cmpl are substantives or proper nouns."""
    cmpl_indiv = []
    for word in cmpl.to_words.filter_in(sp=["subs","nmpr"]):
        cmpl_indiv.append(word.sp[0])
   
    # Handle cases when the individuation is a pronominal suffixe (prsf)    
    simplified_cmpl = []
   
    if len(cmpl.to_words) == 1:
        if cmpl.to_words.sp[0] == "prep":
            if cmpl.to_words.prs not in ["n/a", "absent"]:
                cmpl_indiv.append("prsf")
        else:
            cmpl_indiv.append(cmpl.to_words.sp[0])
        
    # cmpl_indiv empty string if empty    
    if len(cmpl_indiv) == 0:
        cmpl_indiv.append("")
        
    return cmpl_indiv[0]

def find_cmpl_construction(cmpl):
    """Returns the construction of the complement: vc (for verbal complement), 
    prep for prepositional complement and dir_he for complement with directive-he 
    (or a combination of those)."""
    
    cmpl_construction = []
    
    for word in cmpl.to_words:
        if word.filter(sp="prep"):
            cmpl_construction.append("prep")
        elif "H" in word.uvf:
            cmpl_construction.append("dir-he")
            
    if cmpl_construction == []:
        return "vc"
    else:
        return " + ".join(cmpl_construction)
    
    
def find_cmpl_complex(cmpl):
    simplified_cmpl = []
    
    #print(cmpl.to_words.prs)
   
    for word in cmpl.to_words:
        if word.sp[0] != "prep" and word.sp[0] != "art":
            simplified_cmpl.append(word)
    
    if len(simplified_cmpl) == 0 or (len(simplified_cmpl) == 1 and simplified_cmpl[0].to_words.prs[0] in ["n/a", "absent"]):
        cmpl_complex = "simple"
    else:
        cmpl_complex = "complex"
        
    return cmpl_complex

In [62]:
verb = verbs_bhsa[29]
cmpl = find_complements(verb)
anim = find_cmpl_anim(cmpl)
nt = find_cmpl_nametype(cmpl)
indiv = find_cmpl_individuation(cmpl)
constr = find_cmpl_construction(cmpl)

print(cmpl.to_clauses, cmpl, anim, nt, indiv, constr)
#print(find_cmpl_det(cmpl))

CNJM M KL JB>W >LJK >LJK  prsf prsf prep


In [63]:
len(cmpl.to_words)

1

In [64]:
find_cmpl_individuation(cmpl)

'prsf'

In [65]:
find_cmpl_complex(cmpl)
#print(cmpl)

'simple'

In [66]:
cmpl.to_words.filter(sp="prep")

<word_1 ">LJK">

### 3. Generate dataset

In [67]:
items = [] # create an empty list to store all the information for each occ.

       
for verb in verbs_bhsa:
    
    # Add MT as "scroll" for the BHSA
    scroll = "MT"
    verse = verb.to_verses
    dir_he_dss_verse = ""
    sign_info = ""
    
    
    # Filling in the complement column
    complements = find_complements(verb)
    if not complements:
        complements = ["no complement"]
        

    for complement in complements:    
        if complement == "no complement":
            dir_he = cmpl_anim = cmpl_nt = cmpl_det = cmpl_indiv = cmpl_constr = cmpl_complex = ""
            cmpl_lex = "no complement"
          
        else: 
            cmpl_lex = str(" ".join(complement.to_words.lex))
            dir_he = int("H" in complement.to_words.uvf)
            cmpl_anim = find_cmpl_anim(complement)
            cmpl_nt = find_cmpl_nametype(complement)
            cmpl_det = cmpl.det[0]
            cmpl_indiv = find_cmpl_individuation(complement)
            cmpl_constr = find_cmpl_construction(complement)
            cmpl_complex = find_cmpl_complex(complement)
            
    
        # Collect information about the following variables:    
        item = {
            "verb_id": verb.ids[0], 
            "lex": verb.lex[0], 
            "scroll": scroll,
            "book": verb.book[0], 
            "chapter": int(verb.chapter[0]), 
            "verse_num": int(verb.verse[0]),
            "gcons_verb": verb.g_cons[0],
            "gcons_verse": str(verse),
            "gcons_clause": str(verb.to_clauses),
            "subject": str(verb.to_clauses.to_phrases.filter(function="Subj")),
            "complement": str(complement),
            "cmpl_lex": cmpl_lex,
            "cmpl_translation": "",
            "dir_he": dir_he,
            "cmpl_constr": cmpl_constr,
            "cmpl_nt": cmpl_nt,
            "cmpl_anim": cmpl_anim,
            "cmpl_det": cmpl_det,
            "cmpl_indiv": cmpl_indiv,
            "cmpl_complex": cmpl_complex,
            "dir_he_dss": dir_he_dss_verse,
            "sign_info": sign_info,
            "stem": verb.vs[0],
            "tense": verb.vt[0],
            "motion_type": "",
            "comments": "",
        }
        
        if complement != "no complement":
            prepositions = complement.to_words.filter(sp="prep").lex
            n = 0
            for preposition in prepositions:
                n += 1
                item[f"preposition_{n}"] = str(preposition)

        items.append(item)

### 4. Create the dataframe with pandas

In [68]:
df = pd.DataFrame(items).fillna("")
df.sort_values(["book", "chapter", "verse_num"], ascending=[True, True, True], ignore_index=True, inplace=True)

In [69]:
pd.set_option('display.max_columns', None)
#df[df.book == "Exodus"]
#df[df.cmpl_complex == "complex"]

df

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,motion_type,comments,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7
0,391367,JY>[,MT,1_Chronicles,1,12,JY>W,W >T PTRSJM W >T KSLXJM >CR JY>W M CM PLCTJM W...,>CR JY>W M CM PLCTJM,PLCTJM,M CM,MN CM,,0,prep,,,det,,simple,,,qal,perf,,,MN,,,,,,
1,392084,BW>[,MT,1_Chronicles,2,21,B>,W >XR B> XYRWN >L BT MKJR >BJ GL<D W HW> LQXH ...,W >XR B> XYRWN >L BT MKJR >BJ GL<D,XYRWN,>L BT MKJR >BJ GL<D,>L BT/ MKJR/ >B/ GL<D/,,0,prep,"pers,gens topo",inanim,det,subs,complex,,,qal,perf,,,>L,,,,,,
2,392509,JY>[,MT,1_Chronicles,2,53,JY>W,W MCPXWT QRJT_J<RJM H JTRJ W H PWTJ W H CMTJ W...,M >LH JY>W H YR<TJ W H >CT>LJ,H YR<TJ W H >CT>LJ,M >LH,MN >LH,,0,prep,,,det,,simple,,,qal,perf,,,MN,,,,,,
3,392539,BW>[,MT,1_Chronicles,2,55,B>JM,W MCPXWT SPRJM JCBW J<BY TR<TJM CM<TJM FWKTJM ...,H B>JM M XMT >BJ BJT RKB,,M XMT >BJ BJT RKB,MN XMT===/ >B/ BJT/ RKB==/,,0,prep,pers pers,anim anim,det,nmpr,complex,,,qal,ptca,,,MN,,,,,,
4,392967,BW>[,MT,1_Chronicles,4,10,JB>,W JQR> J<BY L >LHJ JFR>L L >MR >M BRK TBRKNJ W...,W JB> >LHJM,>LHJM,no complement,no complement,,,,,,,,,,,hif,wayq,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11251,303935,SWR[,MT,Zephaniah,3,11,>SJR,B JWM H HW> L> TBWCJ M KL <LJLTJK >CR PC<T BJ ...,KJ >Z >SJR M QRBK <LJZJ G>WTK,,M QRBK,MN QRB/,,0,prep,,,det,subs,complex,,,hif,impf,,,MN,,,,,,
11252,303958,XSH[,MT,Zephaniah,3,12,XSW,W HC>RTJ B QRBK <M <NJ W DL W XSW B CM JHWH,W XSW B CM JHWH,,B CM JHWH,B CM/ JHWH/,,0,prep,pers,anim,det,subs,complex,,,qal,perf,,,B,,,,,,
11253,303999,SWR[,MT,Zephaniah,3,15,HSJR,HSJR JHWH MCPVJK PNH >JBK MLK JFR>L JHWH B QRB...,HSJR JHWH MCPVJK,JHWH,no complement,no complement,,,,,,,,,,,hif,perf,,,,,,,,,
11254,304002,PNH[,MT,Zephaniah,3,15,PNH,HSJR JHWH MCPVJK PNH >JBK MLK JFR>L JHWH B QRB...,PNH >JBK,>JBK,no complement,no complement,,,,,,,,,,,piel,perf,,,,,,,,,


In [19]:
filtered_df = df[~((df.preposition_1 == "MN") & (df.preposition_2 == "") & (df.preposition_3 == ""))]
len(filtered_df)

10128

In [58]:
#set(df.cmpl_nt)

### Save the dataframe into a file: the whole BHSA

In [71]:
df.to_csv("data/bhsa_all_verbs.csv", index=False)

In [80]:
#set(df.book)

### Save another csv file without the occurrences of a book + complement, for easier merging with the anntoated dataset

In [81]:
df2 = df[df.book == "1_Samuel"]

In [82]:
df2

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,motion_type,comments,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7
734,141586,<LH[,MT,1_Samuel,1,3,<LH,W <LH H >JC H HW> M <JRW M JMJM JMJMH L HCTXWT...,W <LH H >JC H HW> M <JRW M JMJM JMJMH,H >JC H HW>,M <JRW,MN <JR/,,0,prep,,,det,subs,complex,,,qal,perf,,,MN,,,,,,
735,141672,<LH[,MT,1_Samuel,1,7,<LTH,W KN J<FH CNH B CNH M DJ <LTH B BJT JHWH KN TK...,M DJ <LTH B BJT JHWH,,B BJT JHWH,B BJT/ JHWH/,,0,prep,pers,anim,det,subs,complex,,,qal,infc,,,B,,,,,,
736,141776,<LH[,MT,1_Samuel,1,11,J<LH,W TDR NDR W T>MR JHWH YB>WT >M R>H TR>H B <NJ ...,W MWRH L> J<LH <L R>CW,MWRH,<L R>CW,<L R>C/,,0,prep,,,det,subs,complex,,,qal,impf,,,<L,,,,,,
737,141818,SWR[,MT,1_Samuel,1,14,HSJRJ,W J>MR >LJH <LJ <D MTJ TCTKRJN HSJRJ >T JJNK M...,HSJRJ >T JJNK M <LJK,,M <LJK,MN <L,,0,prep + prep,,,det,,simple,,,hif,impv,,,MN,<L,,,,,
738,141869,HLK[,MT,1_Samuel,1,17,LKJ,W J<N <LJ W J>MR LKJ L CLWM W >LHJ JFR>L JTN >...,LKJ L CLWM,,L CLWM,L CLWM/,,0,prep,,,det,subs,simple,,,qal,impv,,,L,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,160348,BW>[,MT,1_Samuel,31,7,JB>W,W JR>W >NCJ JFR>L >CR B <BR H <MQ W >CR B <BR ...,W JB>W PLCTJM,PLCTJM,no complement,no complement,,,,,,,,,,,qal,wayq,,,,,,,,,
1457,160358,BW>[,MT,1_Samuel,31,8,JB>W,W JHJ M MXRT W JB>W PLCTJM L PCV >T H XLLJM W ...,W JB>W PLCTJM,PLCTJM,no complement,no complement,,,,,,,,,,,qal,wayq,,,,,,,,,
1458,160373,NPL[,MT,1_Samuel,31,8,NPLJM,W JHJ M MXRT W JB>W PLCTJM L PCV >T H XLLJM W ...,NPLJM B HR H GLB<,,B HR H GLB<,B HR/ H GLB</,,0,prep,topo,inanim,det,subs,complex,,,qal,ptca,,,B,,,,,,
1459,160431,HLK[,MT,1_Samuel,31,12,JLKW,W JQWMW KL >JC XJL W JLKW KL H LJLH W JQXW >T ...,W JLKW KL H LJLH,,no complement,no complement,,,,,,,,,,,qal,wayq,,,,,,,,,


In [83]:
df2.to_csv("data/1_samuel_MT_not_annot_with_locatives.csv", index=False)